# Part 6.1 - Prepare Dataframe
---

### Papers Past Topic Modeling
<br/>


Ben Faulks - bmf43@uclive.ac.nz

Xiandong Cai - xca24@uclive.ac.nz

Yujie Cui - ycu23@uclive.ac.nz

In [2]:
import sys
sys.path.insert(0, '../utils')
import pandas as pd
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_colwidth', 120)
from pyspark.sql import functions as F
from pyspark.sql.types import *
from utils import conf_pyspark, load_dataset
from utils_preplot import preplot

# intiate PySpark
sc, spark = conf_pyspark()

sc

[('spark.driver.host', 'x99.hub'),
 ('spark.app.name', 'local'),
 ('spark.driver.port', '34121'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1548305648515'),
 ('spark.driver.memory', '62g'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '6'),
 ('spark.driver.maxResultSize', '4g')]


<SparkContext master=local[*] appName=local>

**This part we generate the dataframe that part 6.2 will be used:**
* dominant topic per year dataframe;
* average topic weight per year dataframe;
* dominant topic per month dataframe (only for WWI analysis);
* average topic weight per month dataframe (only for WWI analysis);

**After training topic models, we got the topic words list (**`topicKeys.txt`**) and the doc-topic matrix (**`docTopics.txt`**). The doc-topic matrix is a docmument based file which means we could connect the topic weights in the matrix and the metadata (date, region, publisher) in the meta dataset, then the combined dataset could apply to accomplish many data mining or statistical works, for instance, the granularity of time can achieve 1 day, which could perform time series analsys. Now we load those data to generate dataframe for analysys and visualization.**

## 1 Prepare Dataframe for Train Set

### 1.1 Load data

**Load metadata ("id", "region" and "date"):**

In [3]:
df_meta = load_dataset('meta', spark).select(F.col('id').alias('id_'), F.col('region'), F.col('date'))

In [3]:
df_meta.limit(5).toPandas().head()

,id_,region,date
0,1854215,Auckland,1862-06-14
1,1854233,Auckland,1863-08-01
2,1854245,Auckland,1863-09-01
3,1854253,Auckland,1863-09-01
4,1854264,Auckland,1863-10-01


**Load topic words list:**

In [4]:
path = r'../models/train/topicKeys.txt'

data_schema = StructType([
    StructField('topic', IntegerType()),
    StructField('weight_', FloatType()),
    StructField('words', StringType())
])

df_topics = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "false")
    .option("delimiter", "\t")
    .schema(data_schema)
    .load(path)
)
topic_number = df_topics.count()

In [5]:
print('Shape of dataframe: ({}, {})'.format(topic_number, len(df_topics.columns)))
df_topics.limit(10).toPandas().head(10)

Shape of dataframe: (200, 3)


,topic,weight_,words
0,0,0.00197,wanted post sell buy anted tel cash good furniture sale
1,1,0.01399,aro moro homo woro sho thoso thorn thoro vory groat
2,2,0.00229,bro lodge grand members secretary district societies officers meeting order
3,3,0.00659,seeds seed sheep grass dip manure clover manures special agents
4,4,0.00444,yds race prize yards handicap sports time sec entrance mile
5,5,0.00274,sellers buyers stock bank zealand sales national exchange pref waihi
6,6,0.01038,court case trial charge guilty jury prisoner sentenced years accused
7,7,0.00440,advertisements office exceeding post half letters ounce insertion postage stamps
8,8,0.01042,hotel proprietor accommodation public spirits wines good rooms travellers brands
9,9,0.00934,naval ships british navy fleet german vessels sea submarine admiral


**Load doc-topic matrix:**

In [5]:
path = r'../models/train/docTopics.txt'

# generate new column names
columns = [str(x) for x in list(range(topic_number))]
columns.insert(0, 'id')
columns.insert(0, 'index')

# load data
df_doctopic = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "true")
    .option("delimiter", "\t")
    .load(path)
)

# change columns name and drop # column which is table index and useless
df_doctopic = df_doctopic.toDF(*columns)

In [7]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (603629, 202)


,index,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,...,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0,1854215,0.000031,0.000223,0.000036,0.000105,0.000071,0.000044,0.000165,0.270538,0.000166,0.000149,0.000214,0.000109,0.000251,0.000104,0.000065,0.000201,0.000138,0.000090,0.000125,0.000261,0.000193,0.000091,0.000118,0.000345,0.000096,0.000221,0.000137,0.000135,0.000168,0.000192,0.000338,0.000064,0.000135,0.000140,0.000146,0.000195,0.000201,0.000124,0.000120,0.000157,0.000113,0.000079,0.000223,0.000136,0.000134,0.000117,0.000096,0.000233,0.000050,0.000153,0.000143,0.000074,0.000094,0.000220,0.000167,0.000154,0.016057,0.000059,...,0.000239,0.000063,0.000164,0.000357,0.000039,0.000116,0.000098,0.000087,0.000109,0.000138,0.127340,0.000043,0.000114,0.000125,0.000164,0.000069,0.000139,0.000127,0.000075,0.000110,0.000029,0.000172,0.000056,0.000125,0.000051,0.000092,0.000024,0.000144,0.000135,0.000033,0.000143,0.000201,0.000147,0.000116,0.000149,0.064899,0.000195,0.000053,0.000155,0.016051,0.000114,0.000115,0.000192,0.000112,0.000160,0.000284,0.000138,0.000165,0.000233,0.000032,0.000325,0.000204,0.000769,0.000143,0.000188,0.000140,0.254716,0.000036,0.000099,0.000093
1,1,1854233,0.000013,0.000093,0.000015,0.000044,0.000030,0.000018,0.006742,0.000029,0.000070,0.000062,0.060148,0.000046,0.000105,0.000044,0.000027,0.140221,0.000058,0.000038,0.000052,0.080187,0.000081,0.000038,0.000049,0.000145,0.000040,0.000093,0.000057,0.000056,0.000071,0.006754,0.000142,0.000027,0.060115,0.000059,0.000061,0.000082,0.000084,0.000052,0.000050,0.000066,0.000048,0.000033,0.000094,0.053442,0.000056,0.046761,0.000040,0.000098,0.000021,0.000064,0.000060,0.000031,0.000039,0.000092,0.006743,0.000065,0.000062,0.000025,...,0.000100,0.000027,0.000069,0.000150,0.000016,0.000049,0.000041,0.000037,0.000046,0.006731,0.120142,0.000018,0.000048,0.000052,0.000069,0.000029,0.000059,0.000053,0.000032,0.000046,0.000012,0.000072,0.000023,0.000052,0.000022,0.000038,0.000010,0.000060,0.000056,0.000014,0.000060,0.000084,0.000061,0.000049,0.000063,0.000528,0.000082,0.000022,0.000065,0.000059,0.000048,0.000048,0.000080,0.000047,0.000067,0.000119,0.000058,0.000069,0.000098,0.000014,0.000136,0.000086,0.000323,0.000060,0.000079,0.000059,0.000066,0.000015,0.000041,0.000039
2,2,1854245,0.000011,0.000080,0.000013,0.000037,0.000025,0.000016,0.000059,0.011398,0.000059,0.000053,0.153613,0.000039,0.130880,0.000037,0.000023,0.091056,0.000049,0.000032,0.000045,0.000093,0.000069,0.000033,0.000042,0.000123,0.000034,0.000079,0.000049,0.000048,0.000060,0.000069,0.000121,0.000023,0.005735,0.000050,0.000052,0.000070,0.000072,0.000044,0.000043,0.000056,0.000041,0.159251,0.000080,0.000049,0.000048,0.000042,0.000034,0.108127,0.000018,0.000055,0.000051,0.000027,0.000033,0.000079,0.000060,0.000055,0.000052,0.000021,...,0.011459,0.000023,0.000059,0.000128,0.000014,0.000042,0.000035,0.000031,0.000039,0.000049,0.000022,0.000015,0.000041,0.000045,0.000059,0.000025,0.000050,0.000045,0.000027,0.000039,0.000010,0.000061,0.000020,0.000045,0.000018,0.000033,0.000009,0.000052,0.000048,0.000012,0.062603,0.000072,0.000052,0.000042,0.000053,0.017510,0.000070,0.000019,0.000055,0.000050,0.000041,0.000041,0.000068,0.000040,0.005744,0.000101,0.000049,0.000059,0.000083,0.005698,0.000116,0.000073,0.005961,0.000051,0.011440,0.000050,0.000057,0.000013,0.000035,0.000033
3,3,1854253,0.000004,0.000029,0.000005,0.000014,0.000009,0.000006,0.000022,0.000009,0.000022,0.000020,0.000028,0.000014,0.000033,0.000014,0.000009,0.121657,0.000018,0.000012,0.000016,0.000034,0.000025,0.000012,0.000016,0.000045,0.000013,0.000029,0.000018,0.000018,0.000022,0.000025,0.021015,0.000008,0.000018,0.000018,0.008408,0.000026,0.000026,0.1

**In above dataframe, "index" column is the row number, "id" column is the sample/document/text id, the same with "id" in dataset, "0" to max topic number columns are the weight of each topic per document.**

### 1.2 Add Dominant Topics Column

**Find dominant topic of each document:**

In [6]:
# https://stackoverflow.com/questions/46819405/how-to-get-the-name-of-column-with-maximum-value-in-pyspark-dataframe

def argmax(cols, *args):
    return [c for c, v in zip(cols, args) if v == max(args)][0]

def add_domtopic(df):
    """
    find the dominant topic of each sample/row/document
    input: dataframe of weight of each topic
    output: the dominant topic number dataframe
    """
    argmax_udf = lambda cols: F.udf(lambda *args: argmax(cols, *args), StringType())
    return (df
            .withColumn('domtopic',
                        argmax_udf(df.columns[2:])(*df.columns[2:]))
            .withColumn('weight', 
                        F.greatest(*[F.col(x) for x in df.columns[2:-1]])))

# add the df_dominant to doc-topic matrix
df_doctopic = add_domtopic(df_doctopic)

In [11]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

,index,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,...,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,domtopic,weight
0,0,1854215,0.000031,0.000223,0.000036,0.000105,0.000071,0.000044,0.000165,0.270538,0.000166,0.000149,0.000214,0.000109,0.000251,0.000104,0.000065,0.000201,0.000138,0.000090,0.000125,0.000261,0.000193,0.000091,0.000118,0.000345,0.000096,0.000221,0.000137,0.000135,0.000168,0.000192,0.000338,0.000064,0.000135,0.000140,0.000146,0.000195,0.000201,0.000124,0.000120,0.000157,0.000113,0.000079,0.000223,0.000136,0.000134,0.000117,0.000096,0.000233,0.000050,0.000153,0.000143,0.000074,0.000094,0.000220,0.000167,0.000154,0.016057,0.000059,...,0.000164,0.000357,0.000039,0.000116,0.000098,0.000087,0.000109,0.000138,0.127340,0.000043,0.000114,0.000125,0.000164,0.000069,0.000139,0.000127,0.000075,0.000110,0.000029,0.000172,0.000056,0.000125,0.000051,0.000092,0.000024,0.000144,0.000135,0.000033,0.000143,0.000201,0.000147,0.000116,0.000149,0.064899,0.000195,0.000053,0.000155,0.016051,0.000114,0.000115,0.000192,0.000112,0.000160,0.000284,0.000138,0.000165,0.000233,0.000032,0.000325,0.000204,0.000769,0.000143,0.000188,0.000140,0.254716,0.000036,0.000099,0.000093,7,0.270538
1,1,1854233,0.000013,0.000093,0.000015,0.000044,0.000030,0.000018,0.006742,0.000029,0.000070,0.000062,0.060148,0.000046,0.000105,0.000044,0.000027,0.140221,0.000058,0.000038,0.000052,0.080187,0.000081,0.000038,0.000049,0.000145,0.000040,0.000093,0.000057,0.000056,0.000071,0.006754,0.000142,0.000027,0.060115,0.000059,0.000061,0.000082,0.000084,0.000052,0.000050,0.000066,0.000048,0.000033,0.000094,0.053442,0.000056,0.046761,0.000040,0.000098,0.000021,0.000064,0.000060,0.000031,0.000039,0.000092,0.006743,0.000065,0.000062,0.000025,...,0.000069,0.000150,0.000016,0.000049,0.000041,0.000037,0.000046,0.006731,0.120142,0.000018,0.000048,0.000052,0.000069,0.000029,0.000059,0.000053,0.000032,0.000046,0.000012,0.000072,0.000023,0.000052,0.000022,0.000038,0.000010,0.000060,0.000056,0.000014,0.000060,0.000084,0.000061,0.000049,0.000063,0.000528,0.000082,0.000022,0.000065,0.000059,0.000048,0.000048,0.000080,0.000047,0.000067,0.000119,0.000058,0.000069,0.000098,0.000014,0.000136,0.000086,0.000323,0.000060,0.000079,0.000059,0.000066,0.000015,0.000041,0.000039,115,0.146925
2,2,1854245,0.000011,0.000080,0.000013,0.000037,0.000025,0.000016,0.000059,0.011398,0.000059,0.000053,0.153613,0.000039,0.130880,0.000037,0.000023,0.091056,0.000049,0.000032,0.000045,0.000093,0.000069,0.000033,0.000042,0.000123,0.000034,0.000079,0.000049,0.000048,0.000060,0.000069,0.000121,0.000023,0.005735,0.000050,0.000052,0.000070,0.000072,0.000044,0.000043,0.000056,0.000041,0.159251,0.000080,0.000049,0.000048,0.000042,0.000034,0.108127,0.000018,0.000055,0.000051,0.000027,0.000033,0.000079,0.000060,0.000055,0.000052,0.000021,...,0.000059,0.000128,0.000014,0.000042,0.000035,0.000031,0.000039,0.000049,0.000022,0.000015,0.000041,0.000045,0.000059,0.000025,0.000050,0.000045,0.000027,0.000039,0.000010,0.000061,0.000020,0.000045,0.000018,0.000033,0.000009,0.000052,0.000048,0.000012,0.062603,0.000072,0.000052,0.000042,0.000053,0.017510,0.000070,0.000019,0.000055,0.000050,0.000041,0.000041,0.000068,0.000040,0.005744,0.000101,0.000049,0.000059,0.000083,0.005698,0.000116,0.000073,0.005961,0.000051,0.011440,0.000050,0.000057,0.000013,0.000035,0.000033,41,0.159251
3,3,1854253,0.000004,0.000029,0.000005,0.000014,0.000009,0.000006,0.000022,0.000009,0.000022,0.000020,0.000028,0.000014,0.000033,0.000014,0.000009,0.121657,0.000018,0.000012,0.000016,0.000034,0.000025,0.000012,0.000016,0.000045,0.000013,0.000029,0.000018,0.000018,0.000022,0.000025,0.021015,0.000008,0.000018,0.000018,0.008408,0.000026,0.000026,0.186656,0.00

### 1.3 Add Metadata Columns

**Here we only add "region" and "date" column as metadata, the accuracy of time could achieve "day" level, using the time series features we could convert "date" to year base or month base etc. depending on the need.**

In [14]:
df_doctopic = (df_doctopic
               .join(df_meta, df_doctopic.id == df_meta.id_)
               .withColumn('year', F.date_format('date', 'yyyy'))
               .drop('id_')
               .drop('date')
               .orderBy('index'))

In [15]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

,index,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,...,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,domtopic,weight,region,year
0,0,1854215,0.000031,0.000223,0.000036,0.000105,0.000071,0.000044,0.000165,0.270538,0.000166,0.000149,0.000214,0.000109,0.000251,0.000104,0.000065,0.000201,0.000138,0.000090,0.000125,0.000261,0.000193,0.000091,0.000118,0.000345,0.000096,0.000221,0.000137,0.000135,0.000168,0.000192,0.000338,0.000064,0.000135,0.000140,0.000146,0.000195,0.000201,0.000124,0.000120,0.000157,0.000113,0.000079,0.000223,0.000136,0.000134,0.000117,0.000096,0.000233,0.000050,0.000153,0.000143,0.000074,0.000094,0.000220,0.000167,0.000154,0.016057,0.000059,...,0.000039,0.000116,0.000098,0.000087,0.000109,0.000138,0.127340,0.000043,0.000114,0.000125,0.000164,0.000069,0.000139,0.000127,0.000075,0.000110,0.000029,0.000172,0.000056,0.000125,0.000051,0.000092,0.000024,0.000144,0.000135,0.000033,0.000143,0.000201,0.000147,0.000116,0.000149,0.064899,0.000195,0.000053,0.000155,0.016051,0.000114,0.000115,0.000192,0.000112,0.000160,0.000284,0.000138,0.000165,0.000233,0.000032,0.000325,0.000204,0.000769,0.000143,0.000188,0.000140,0.254716,0.000036,0.000099,0.000093,7,0.270538,Auckland,1862
1,1,1854233,0.000013,0.000093,0.000015,0.000044,0.000030,0.000018,0.006742,0.000029,0.000070,0.000062,0.060148,0.000046,0.000105,0.000044,0.000027,0.140221,0.000058,0.000038,0.000052,0.080187,0.000081,0.000038,0.000049,0.000145,0.000040,0.000093,0.000057,0.000056,0.000071,0.006754,0.000142,0.000027,0.060115,0.000059,0.000061,0.000082,0.000084,0.000052,0.000050,0.000066,0.000048,0.000033,0.000094,0.053442,0.000056,0.046761,0.000040,0.000098,0.000021,0.000064,0.000060,0.000031,0.000039,0.000092,0.006743,0.000065,0.000062,0.000025,...,0.000016,0.000049,0.000041,0.000037,0.000046,0.006731,0.120142,0.000018,0.000048,0.000052,0.000069,0.000029,0.000059,0.000053,0.000032,0.000046,0.000012,0.000072,0.000023,0.000052,0.000022,0.000038,0.000010,0.000060,0.000056,0.000014,0.000060,0.000084,0.000061,0.000049,0.000063,0.000528,0.000082,0.000022,0.000065,0.000059,0.000048,0.000048,0.000080,0.000047,0.000067,0.000119,0.000058,0.000069,0.000098,0.000014,0.000136,0.000086,0.000323,0.000060,0.000079,0.000059,0.000066,0.000015,0.000041,0.000039,115,0.146925,Auckland,1863
2,2,1854245,0.000011,0.000080,0.000013,0.000037,0.000025,0.000016,0.000059,0.011398,0.000059,0.000053,0.153613,0.000039,0.130880,0.000037,0.000023,0.091056,0.000049,0.000032,0.000045,0.000093,0.000069,0.000033,0.000042,0.000123,0.000034,0.000079,0.000049,0.000048,0.000060,0.000069,0.000121,0.000023,0.005735,0.000050,0.000052,0.000070,0.000072,0.000044,0.000043,0.000056,0.000041,0.159251,0.000080,0.000049,0.000048,0.000042,0.000034,0.108127,0.000018,0.000055,0.000051,0.000027,0.000033,0.000079,0.000060,0.000055,0.000052,0.000021,...,0.000014,0.000042,0.000035,0.000031,0.000039,0.000049,0.000022,0.000015,0.000041,0.000045,0.000059,0.000025,0.000050,0.000045,0.000027,0.000039,0.000010,0.000061,0.000020,0.000045,0.000018,0.000033,0.000009,0.000052,0.000048,0.000012,0.062603,0.000072,0.000052,0.000042,0.000053,0.017510,0.000070,0.000019,0.000055,0.000050,0.000041,0.000041,0.000068,0.000040,0.005744,0.000101,0.000049,0.000059,0.000083,0.005698,0.000116,0.000073,0.005961,0.000051,0.011440,0.000050,0.000057,0.000013,0.000035,0.000033,41,0.159251,Auckland,1863
3,3,1854253,0.000004,0.000029,0.000005,0.000014,0.000009,0.000006,0.000022,0.000009,0.000022,0.000020,0.000028,0.000014,0.000033,0.000014,0.000009,0.121657,0.000018,0.000012,0.000016,0.000034,0.000025,0.000012,0.000016,0.000045,0.000013,0.000029,0.000018,0.000018,0.000022,0.000025,0.021015,0.000008,0.000018,0.000018,0.008408,0.000026,0.000026,0.186656,0.000016,0.0

### 1.4 Document - Dominant Topics Dataframe

**It is hard to plot the doc-topic matrix (high dimension), we need transform it to extract or reduce features. First we generate dominant topics dataframe which could be used to reveal the relationship between dominant topics and region/year.**

In [16]:
df_docdomtopic = (df_doctopic
                  .join(df_topics, df_doctopic.domtopic == df_topics.topic)
                  .select(F.col('id'), 
                          F.col('region'), 
                          F.col('year'), 
                          F.col('domtopic'), 
                          F.col('weight'), 
                          F.col('words'))
                  .orderBy('id'))

df_docdomtopic.cache()

In [ ]:
print('Shape of dataframe: ({}, {})'.format(df_docdomtopic.count(), len(df_docdomtopic.columns)))

df_docdomtopic.limit(5).toPandas().head()

**Save the dataframe for later use:**

In [13]:
path = r'../models/train/'

df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic

In [14]:
%%bash -s "$path"

cat $1/*.csv.gz > $1/domTopics.csv.gz

gunzip $1/domTopics.csv.gz

### 1.5 Average Weight Topics Dataframe

**Beside dominant topics dataframe, we could calculate average weight of each topic in a year and create dataframe for it, which could be used to reveal the weight variety of each topic as time goes on. The weights of each topic were already scaled to 0-1 by default, so the sum of average weight of each year is 1, we do not need to scale it.**

In [8]:
df_avgweight = (df_doctopic
                .drop('index')
                .drop('id')
                .drop('dominant')
                .drop('region')
                .drop('weight')
                .groupBy('year')
                .avg()
                .orderBy('year'))

In [9]:
df_avgweight.limit(5).toPandas().head()

,year,avg(0),avg(1),avg(2),avg(3),avg(4),avg(5),avg(6),avg(7),avg(8),...,avg(240),avg(241),avg(242),avg(243),avg(244),avg(245),avg(246),avg(247),avg(248),avg(249)
0,1840,0.000081,0.008279,0.000056,0.023775,0.000042,0.000384,0.000063,0.013187,0.000047,...,0.000144,0.000070,0.000091,0.000054,0.000061,0.000109,0.000089,0.000060,0.011574,0.000036
1,1841,0.000161,0.011706,0.004822,0.030666,0.000254,0.005892,0.000126,0.007246,0.000095,...,0.000287,0.000140,0.000182,0.000107,0.000122,0.001599,0.001406,0.000119,0.107492,0.000072
2,1842,0.000116,0.003160,0.002450,0.023355,0.000061,0.012671,0.000091,0.000454,0.000068,...,0.001251,0.000101,0.000131,0.000077,0.000088,0.000489,0.001364,0.011884,0.101871,0.000052
3,1843,0.000675,0.000822,0.003062,0.044746,0.000048,0.002650,0.000071,0.000095,0.000136,...,0.004082,0.000938,0.000852,0.000275,0.000069,0.000389,0.004093,0.000067,0.047911,0.000041
4,1844,0.000519,0.000147,0.000698,0.050104,0.003802,0.009987,0.000114,0.000153,0.000086,...,0.000260,0.000127,0.000336,0.000097,0.000111,0.000397,0.001362,0.000108,0.005803,0.000065


**check years are identical with dataset:**

In [17]:
print('Year range of average weight dataframe:', 
      df_avgweight.select(F.min('year'), F.max('year')).first(), 
      '\nYear number:',
      df_avgweight.select('year').distinct().count())
print('Year range of doc-topic      dataframe:', 
      df_doctopic.select(F.min('year'), F.max('year')).first(), 
      '\nYear number:',
      df_doctopic.select('year').distinct().count())

Year range of average weight dataframe: Row(min(year)='1840', max(year)='1945') 
Year number: 103
Year range of doc-topic      dataframe: Row(min(year)='1840', max(year)='1945') 
Year number: 103


**The dimension of the avrage weight dataframe is small (topic_n\*year_n), so we directly save the dataframe using Pandas:**

In [10]:
path = r'./train/avgWeight/'

df_avgweight.toPandas().to_csv(path+'avgWeight.csv', header=False, index=False, encoding='utf-8')

## 2 Prepare Dataframe for Subset

**The process for subset is the same with for full dataset, we wrapped the process to a function and call it for each subset.**

### 2.1 By Range of Time

**Generate dataframes:**

In [19]:
file = r'../5-model/model_wwi/docTopicsInfer.txt'

df_dominant, df_avgweight = preplot(file, df_meta, df_topics, spark)

**Save dataframes:**

In [20]:
path = r'./wwi/avgWeight/'
df_avgweight.toPandas().to_csv(path+'avgWeight.csv', header=False, index=False, encoding='utf-8')

path = r'./wwi/domTopics'
df_dominant.write.csv(path, mode='overwrite', compression='gzip')

**Convert multi files to a csv file:**

In [21]:
%%bash -s "$path"

cat $1/*.csv.gz > $1/domTopics.csv.gz

gunzip $1/domTopics.csv.gz

### 2.2 By Region

**Generate dataframes:**

In [22]:
file = r'../5-model/model_regions/docTopicsInfer.txt'

df_dominant, df_avgweight = preplot(file, df_meta, df_topics, spark)

**Save dataframes:**

In [23]:
path = r'./regions/avgWeight/'
df_avgweight.toPandas().to_csv(path+'avgWeight.csv', header=False, index=False, encoding='utf-8')

path = r'./regions/domTopics'
df_dominant.write.csv(path, mode='overwrite', compression='gzip')

**Convert multi files to a csv file:**

In [24]:
%%bash -s "$path"

cat $1/*.csv.gz > $1/domTopics.csv.gz

gunzip $1/domTopics.csv.gz

### 2.3 By Label

**Generate dataframes:**

In [25]:
file = r'../5-model/model_ads/docTopicsInfer.txt'

df_dominant, df_avgweight = preplot(file, df_meta, df_topics, spark)

**Save dataframes:**

In [26]:
path = r'./ads/avgWeight/'
df_avgweight.toPandas().to_csv(path+'avgWeight.csv', header=False, index=False, encoding='utf-8')

path = r'./ads/domTopics'
df_dominant.write.csv(path, mode='overwrite', compression='gzip')

**Convert multi files to a csv file:**

In [27]:
%%bash -s "$path"

cat $1/*.csv.gz > $1/domTopics.csv.gz

gunzip $1/domTopics.csv.gz

---